# Merge two tables

## Description

Our database administrator was contacted to merge the data from a new staging table, with updated, deleted and inserted values, into an old production table.
Values which haven´t been changed should stay the same.

## Requirements

In this example we are going to work with the AdventureWorks database every SQL-Professional knows.

## Preparation

We need to prepare two tables with the same structure and different data.

In [7]:
-- Create two tables with data to merge

-- Use AdventureWorks database
USE AdventureWorks2017;

-- Drop tables if exists
DROP TABLE IF EXISTS Production.Beer_Prod;
DROP TABLE IF EXISTS Production.Beer_Staging;

-- create new tables
CREATE TABLE Production.Beer_Prod (
    id INT PRIMARY KEY,
    brand VARCHAR(255) NOT NULL,
    amount DECIMAL(10, 2)
);

CREATE TABLE Production.Beer_Staging (
    id INT PRIMARY KEY,
    brand VARCHAR(255) NOT NULL,
    amount DECIMAL(10, 2)
);

-- fill tables with data
INSERT INTO Production.Beer_Prod VALUES (1, 'Nothhaft Urrr-Hellll', 1000);
INSERT INTO Production.Beer_Prod VALUES (2, 'Langbraeu Spezial', 1500);
INSERT INTO Production.Beer_Prod VALUES (3, 'Gambrinus Zoigl', 1280);
INSERT INTO Production.Beer_Prod VALUES (4, 'Masel Wesse', 100000);
INSERT INTO Production.Beer_Prod VALUES (1000, 'Graefs Pflaume', 1000);

INSERT INTO Production.Beer_Staging VALUES (1, 'Nothhaft Ur-Hell', 1000);
INSERT INTO Production.Beer_Staging VALUES (2, 'Langbraeu Spezial', 1500);
INSERT INTO Production.Beer_Staging VALUES (3, 'Gambrinus Zoigl', 1300);
INSERT INTO Production.Beer_Staging VALUES (4, 'Maisels Weisse', 2000);
INSERT INTO Production.Beer_Staging VALUES (5, 'Langbraeu Spezial', 1500);

Now lets select our data and think about what to do.

In the end the production table should contain all values from staging.

In [8]:
-- select created data

-- Use AdventureWorks database
USE AdventureWorks2017;

-- select all entries
SELECT * FROM Production.Beer_Prod;
SELECT * FROM Production.Beer_Staging;

## Solution

### What to do with the data

Lets think which id´s are different and what to do with them. For a better understanding first lets define which table is source and which is target.

**Production.Beer_Staging** is ``source`` and **Production.Beer_Prod** is ``target``.

Now there are different cases for each id:

- The source table has rows that do not exist in target table. In this case, you have to ``insert`` rows from source to target.
- The target table has rows that do not exist in source table. In this case, you have to ``delete`` rows from target.
- The source table has rows with the same id as in target table. However, these rows have different values, you need to ``update`` the target with the source values.



![title](img_merge/Merge_Overview.png)

You can solve this by using 3 different statements. SQL Server provides the ``MERGE`` statement that allows to perfom this at the same time.

The following lines show the syntax of this statement.

In [0]:
MERGE target USING source
ON merge_condition
WHEN MATCHED
    THEN update_statement
WHEN NOT MATCHED BY TARGET
    THEN insert_statement
WHEN NOT MATCHED BY SOURCE
    THEN DELETE;

Here we have to add that the 3 results are optional. You can perform a merge statement only with ``MATCHED`` and ``NOT MATCHED BY TARGET`` or other combinations too.

### Merge our example data

Now we are merging our example data.

In [9]:
-- merge our tables

-- Use AdventureWorks database
USE AdventureWorks2017;

-- merge our data
MERGE Production.Beer_Prod prod
    USING Production.Beer_Staging stage
ON (prod.id = stage.id)
WHEN MATCHED
    THEN UPDATE SET
        prod.brand = stage.brand,
        prod.amount = stage.amount
WHEN NOT MATCHED BY TARGET
    THEN INSERT VALUES (stage.id, stage.brand, stage.amount)
WHEN NOT MATCHED BY SOURCE
    THEN DELETE;

Lets select our data to verify the result.

In [10]:
-- merge our tables

-- Use AdventureWorks database
USE AdventureWorks2017;

-- select data
SELECT * FROM Production.Beer_Prod;

## Cleanup

In [11]:
--cleanup

USE AdventureWorks2017;

DROP TABLE Production.Beer_Prod;
DROP TABLE Production.Beer_Staging;